In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import torch
from collections import Counter

## Labeled transcripts example

Just looking at the Fastai subset.

In [6]:
transcripts=pd.read_json('~/youtube_captions_corrections/data/transcripts/en/labeled_transcripts/Jeremy_Howard.json')
transcripts.columns   # DataFrame index is the `video_id`

Index(['video_titles', 'playlist_ids', 'channel_ids', 'autogen', 'manual',
       'autogen_text', 'manual_text', 'diffs', 'common_to_both_seq',
       'is_autogen_unique', 'is_manual_unique', 'autogen_seq', 'manual_seq',
       'manual_addl_rep'],
      dtype='object')

In [7]:
transcripts.head()

,video_titles,playlist_ids,channel_ids,autogen,manual,autogen_text,manual_text,diffs,common_to_both_seq,is_autogen_unique,is_manual_unique,autogen_seq,manual_seq,manual_addl_rep
_QUEXsHfsA0,Lesson 1 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'so hello everybody and welcome to d...,[{'text': 'So hello everybody and welcome to D...,so hello everybody and welcome to deep learnin...,So hello everybody and welcome to Deep Learnin...,"[{'-': ['so'], '+': ['So']}, hello, everybody,...","[, hello, everybody, and, welcome, to, , , for...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[so, , , , , , deep, learning, , coders, lesso...","[So, , , , , , Deep Learning, Deep Learning, ,...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 3, 3, 3, 0, 0, ..."
BvHmRx14HQ8,Lesson 2 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'so uh hello everybody and welcome b...,"[{'text': 'So, hello everybody, and welcome ba...",so uh hello everybody and welcome back to prac...,"So, hello everybody, and welcome back to Pract...","[{'-': ['so', 'uh'], '+': ['So,']}, hello, {'-...","[, , hello, , and, welcome, back, to, , , , fo...","[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[so, uh, , everybody, , , , , practical, deep,...","[So,, So,, , everybody,, , , , , Practical Dee...","[1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 1, 0, ..."
5L3Ao5KuCC4,Lesson 3 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'oh hello and welcome to lesson thre...,"[{'text': 'So hello, and welcome to Lesson 3 o...",oh hello and welcome to lesson three of practi...,"So hello, and welcome to Lesson 3 of Practical...","[{'-': ['oh', 'hello'], '+': ['So', 'hello,']}...","[, , and, welcome, to, , , of, , , , for, , , ...","[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[oh, hello, , , , lesson, three, , practical, ...","[So hello,, So hello,, , , , Lesson 3, Lesson ...","[1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 2, 0, 1, 1, 0, ..."
p50s63nPq9I,Lesson 4 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'welcome back and here is lesson or ...,[{'text': 'Welcome back and here is lesson 4 w...,welcome back and here is lesson or which is wh...,Welcome back and here is lesson 4 which is\nwh...,"[{'-': ['welcome'], '+': ['Welcome']}, back, a...","[, back, and, here, is, lesson, , which, is, w...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[welcome, , , , , , or, , , , , , , , , , , , ...","[Welcome, , , , , , 4, , , , , , , , , , , , ,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
krIVOb23EH8,Lesson 5 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'welcome to lesson five and we'll be...,[{'text': 'Welcome to lesson five and we'll be...,welcome to lesson five and we'll be talking ab...,Welcome to lesson five and we'll be talking\na...,"[{'-': ['welcome'], '+': ['Welcome']}, to, les...","[, to, lesson, five, and, we'll, be, talking, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[welcome, , , , , , , , , , , , , , , , , , , ...","[Welcome, , , , , , , , , , , , , , , , , , , ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Video metadata

In [8]:
transcripts[['video_titles', 'playlist_ids', 'channel_ids']]

,video_titles,playlist_ids,channel_ids
_QUEXsHfsA0,Lesson 1 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
BvHmRx14HQ8,Lesson 2 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
5L3Ao5KuCC4,Lesson 3 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
p50s63nPq9I,Lesson 4 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
krIVOb23EH8,Lesson 5 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
cX30jxMNBUw,Lesson 6 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
VEG5xT5gAHc,Lesson 7 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
WjnwWeGjZcM,Lesson 8 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ
XfoYk_Z5AkI,Lesson 1: Deep Learning 2019 - Image classific...,PLfYUBJiXbdtSIJb-Qd3pw0cqCbkGeS0xn,UCX7Y2qWriXpqocG97SFW2OQ
ccMHJeQU4Qw,Lesson 2: Deep Learning 2019 - Data cleaning a...,PLfYUBJiXbdtSIJb-Qd3pw0cqCbkGeS0xn,UCX7Y2qWriXpqocG97SFW2OQ


### Transcript text and metadata

In [9]:
transcripts[['autogen', 'manual','autogen_text', 'manual_text', 'diffs',]]

,autogen,manual,autogen_text,manual_text,diffs
_QUEXsHfsA0,[{'text': 'so hello everybody and welcome to d...,[{'text': 'So hello everybody and welcome to D...,so hello everybody and welcome to deep learnin...,So hello everybody and welcome to Deep Learnin...,"[{'-': ['so'], '+': ['So']}, hello, everybody,..."
BvHmRx14HQ8,[{'text': 'so uh hello everybody and welcome b...,"[{'text': 'So, hello everybody, and welcome ba...",so uh hello everybody and welcome back to prac...,"So, hello everybody, and welcome back to Pract...","[{'-': ['so', 'uh'], '+': ['So,']}, hello, {'-..."
5L3Ao5KuCC4,[{'text': 'oh hello and welcome to lesson thre...,"[{'text': 'So hello, and welcome to Lesson 3 o...",oh hello and welcome to lesson three of practi...,"So hello, and welcome to Lesson 3 of Practical...","[{'-': ['oh', 'hello'], '+': ['So', 'hello,']}..."
p50s63nPq9I,[{'text': 'welcome back and here is lesson or ...,[{'text': 'Welcome back and here is lesson 4 w...,welcome back and here is lesson or which is wh...,Welcome back and here is lesson 4 which is\nwh...,"[{'-': ['welcome'], '+': ['Welcome']}, back, a..."
krIVOb23EH8,[{'text': 'welcome to lesson five and we'll be...,[{'text': 'Welcome to lesson five and we'll be...,welcome to lesson five and we'll be talking ab...,Welcome to lesson five and we'll be talking\na...,"[{'-': ['welcome'], '+': ['Welcome']}, to, les..."
cX30jxMNBUw,[{'text': 'hi everybody and welcome to lesson ...,[{'text': 'Hi everybody and welcome to Lesson ...,hi everybody and welcome to lesson six where w...,"Hi everybody and welcome to Lesson 6, where\nw...","[{'-': ['hi'], '+': ['Hi']}, everybody, and, w..."
VEG5xT5gAHc,[{'text': 'hi everybody and welcome to lesson ...,[{'text': 'Hi everybody and welcome to lesson ...,hi everybody and welcome to lesson 7 we're goi...,Hi everybody and welcome to lesson 7! We're\ng...,"[{'-': ['hi'], '+': ['Hi']}, everybody, and, w..."
WjnwWeGjZcM,[{'text': 'hi everybody and welcome to lesson ...,[{'text': 'Hi everybody and welcome to lesson ...,hi everybody and welcome to lesson eight the l...,"Hi everybody and welcome to lesson eight,\nthe...","[{'-': ['hi'], '+': ['Hi']}, everybody, and, w..."
XfoYk_Z5AkI,[{'text': 'okay so welcome practical deep lear...,"[{'text': 'Okay', 'start': 0.159, 'duration': ...",okay so welcome practical deep learning for co...,Okay so Welcome Practical deep learning for co...,"[{'-': ['okay'], '+': ['Okay']}, so, {'-': ['w..."
ccMHJeQU4Qw,[{'text': 'welcome to lesson two where we're g...,[{'text': 'Welcome to Lesson 2 where we're goi...,welcome to lesson two where we're going to be ...,Welcome to Lesson 2 where we're going to be ta...,"[{'-': ['welcome'], '+': ['Welcome']}, to, {'-..."


### Label data

In [10]:
transcripts[['is_autogen_unique', 'is_manual_unique', 'autogen_seq', 'manual_seq', 'manual_addl_rep']]

,is_autogen_unique,is_manual_unique,autogen_seq,manual_seq,manual_addl_rep
_QUEXsHfsA0,"[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[so, , , , , , deep, learning, , coders, lesso...","[So, , , , , , Deep Learning, Deep Learning, ,...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 3, 3, 3, 0, 0, ..."
BvHmRx14HQ8,"[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[so, uh, , everybody, , , , , practical, deep,...","[So,, So,, , everybody,, , , , , Practical Dee...","[1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 1, 0, ..."
5L3Ao5KuCC4,"[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[oh, hello, , , , lesson, three, , practical, ...","[So hello,, So hello,, , , , Lesson 3, Lesson ...","[1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 2, 0, 1, 1, 0, ..."
p50s63nPq9I,"[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[welcome, , , , , , or, , , , , , , , , , , , ...","[Welcome, , , , , , 4, , , , , , , , , , , , ,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
krIVOb23EH8,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[welcome, , , , , , , , , , , , , , , , , , , ...","[Welcome, , , , , , , , , , , , , , , , , , , ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
cX30jxMNBUw,"[2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[hi, , , , , lesson, six, , , , , , , , , , , ...","[Hi, , , , , Lesson 6,, Lesson 6,, , , , , , ,...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
VEG5xT5gAHc,"[2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[hi, , , , , , 7, we're, , , , , , , , , , , ,...","[Hi, , , , , , 7! We're, 7! We're, , , , , , ,...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
WjnwWeGjZcM,"[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[hi, , , , , , eight, , , , , , , , , course, ...","[Hi, , , , , , eight,, , , , , , , , , course....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
XfoYk_Z5AkI,"[2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[okay, , welcome, practical, , , , , , , , , ,...","[Okay, , Welcome Practical, Welcome Practical,...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ccMHJeQU4Qw,"[2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[welcome, , lesson, two, , , , , , , , , , , ,...","[Welcome, , Lesson 2, Lesson 2, , , , , , , , ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


From `label_diff_targets()`
```
BOTH_AGREE = 0
BOTH_DIFFER = 2
AUTOGEN_INSERT = 1
MANUAL_INSERT = -1
```

## Reconstruction of transcripts from labels

In [11]:
def get_autogen_reconstruct(example):
    autogen_reconstruct = []
    labels = example.is_autogen_unique
    for idx, label in enumerate(labels):
        if label == 0:
            autogen_reconstruct.append(example.common_to_both_seq[idx])
        if label == 1 or label == 2:
            autogen_reconstruct.append(example.autogen_seq[idx])
        if label == -1:
            autogen_reconstruct.append("")
    return autogen_reconstruct

In [12]:

def get_manual_reconstruct(example):
    manual_reconstruct = []
    labels = example.is_manual_unique
    i = 0
    while i < len(labels):
        if labels[i] == 0:
            manual_reconstruct.append(example.common_to_both_seq[i])
        elif labels[i] == -1:
            manual_reconstruct.extend(example.manual_seq[i].split())
        elif labels[i] == 2:
            manual_reconstruct.extend(example.manual_seq[i].split())
            i+=example.manual_addl_rep[i]
        i+=1
    return manual_reconstruct 


In [13]:
example = transcripts.iloc[0]

In [14]:
autogen_recon = (get_autogen_reconstruct(example))
autogen_text = example.autogen_text.split()
assert ' '.join(autogen_recon).split() == ' '.join(autogen_text).split()
autogen_recon[:10]

['so',
 'hello',
 'everybody',
 'and',
 'welcome',
 'to',
 'deep',
 'learning',
 'for',
 'coders']

In [15]:
manual_recon = get_manual_reconstruct(example)
manual_text = example.manual_text.split()
assert manual_recon == manual_text
manual_recon[:10]

['So',
 'hello',
 'everybody',
 'and',
 'welcome',
 'to',
 'Deep',
 'Learning',
 'for',
 'Coders,']

## Additional label creation

Many of the labeled differences between the autogen and manual transcripts will not be useful for most tasks. But hopefully the labeling schema is flexible enough that people can use it to create their own more specific labels.

Below is an example of adding labels for simple (removing case and punctuation) single token differences

In [16]:
def add_simple_single_token_diff_labels(t, new_label_name, label):
    REP_TARGET = 0 # looking for only a single token differences (no repetitions)
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    
    new_labels = [0]*len(t.autogen_seq)
    for idx in range(len(t.autogen_seq)):
        if t.is_autogen_unique[idx] == 2 and t.manual_addl_rep[idx] == REP_TARGET:
            auto_token = t.autogen_seq[idx]
            man_token = t.manual_seq[idx]

            auto_token = ' '.join(tokenizer.tokenize(auto_token.strip())).lower()
            man_token = ' '.join(tokenizer.tokenize(man_token.strip())).lower()

            if (
                auto_token != man_token  # still different after above simplifications
                and len(man_token.split()) == 1  # only a single token difference
                and not re.match("\d+", man_token)  # no digits in either: 
                and not re.match("\d+", auto_token)  # e.g. `2` <-> `two` is a common diff
            ):
                new_labels[idx]=label

    t[new_label_name] = new_labels
    return t

In [17]:
SINGLE_DIFF_LABEL = 1

In [18]:
transcripts = transcripts.apply(
    add_simple_single_token_diff_labels, 
    axis=1, 
    args=('is_single_diff', SINGLE_DIFF_LABEL)
)

In [19]:
transcripts.head(3)

,video_titles,playlist_ids,channel_ids,autogen,manual,autogen_text,manual_text,diffs,common_to_both_seq,is_autogen_unique,is_manual_unique,autogen_seq,manual_seq,manual_addl_rep,is_single_diff
_QUEXsHfsA0,Lesson 1 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'so hello everybody and welcome to d...,[{'text': 'So hello everybody and welcome to D...,so hello everybody and welcome to deep learnin...,So hello everybody and welcome to Deep Learnin...,"[{'-': ['so'], '+': ['So']}, hello, everybody,...","[, hello, everybody, and, welcome, to, , , for...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, ...","[so, , , , , , deep, learning, , coders, lesso...","[So, , , , , , Deep Learning, Deep Learning, ,...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 3, 3, 3, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BvHmRx14HQ8,Lesson 2 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'so uh hello everybody and welcome b...,"[{'text': 'So, hello everybody, and welcome ba...",so uh hello everybody and welcome back to prac...,"So, hello everybody, and welcome back to Pract...","[{'-': ['so', 'uh'], '+': ['So,']}, hello, {'-...","[, , hello, , and, welcome, back, to, , , , fo...","[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, ...","[so, uh, , everybody, , , , , practical, deep,...","[So,, So,, , everybody,, , , , , Practical Dee...","[1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5L3Ao5KuCC4,Lesson 3 - Deep Learning for Coders (2020),PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM,UCX7Y2qWriXpqocG97SFW2OQ,[{'text': 'oh hello and welcome to lesson thre...,"[{'text': 'So hello, and welcome to Lesson 3 o...",oh hello and welcome to lesson three of practi...,"So hello, and welcome to Lesson 3 of Practical...","[{'-': ['oh', 'hello'], '+': ['So', 'hello,']}...","[, , and, welcome, to, , , of, , , , for, , , ...","[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, ...","[oh, hello, , , , lesson, three, , practical, ...","[So hello,, So hello,, , , , Lesson 3, Lesson ...","[1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 2, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


#### Looking at some example simple single diff labels. 
- First line printed below is autogen sequences with the labeled `*simple single diff*` highlighted.
- Second line is autogen sequences with the corresponding `**manual sequence token**` label inserted.

Some of the diffs highlighted below do seem interesting and learnable by a trained language model, while others are less interesting and could ideally be further filtered out by improved additional label creation.

In [20]:
row = transcripts.iloc[0]
new_labels = torch.tensor(row.is_single_diff)
idxs = torch.where(new_labels == SINGLE_DIFF_LABEL)[0]

for idx in idxs[:20]:
    autogen_seq_recon = get_autogen_reconstruct(row)
    manual_seq_recon = get_manual_reconstruct(row)
    
    autogen_seq_recon[idx] = f"*{autogen_seq_recon[idx]}*"
    print(' '.join(autogen_seq_recon[idx-5:idx+5]))
    autogen_seq_recon[idx] = f"**{example.manual_seq[idx]}**"
    print(' '.join(autogen_seq_recon[idx-5:idx+5]))
    print()

while now we've actually finally *gotten* to the point where
while now we've actually finally **got** to the point where

about to the point that *Sylvia* and I have actually
about to the point that **Sylvain** and I have actually

software from scratch called the *faster* library version 2 we've
software from scratch called the **fastai** library version 2 we've

as you see it's a *laughter* but by the time
as you see it's a **draft,** but by the time

is we well I mentioned *why* important part of the
is we well I mentioned **one** important part of the

Rachel Thomas and so maybe *richer* you can come and
Rachel Thomas and so maybe **Rachel** you can come and

the founding director of the *Centre* for applied data ethics
the founding director of the **Center** for applied data ethics

and I'll be the voice *to* hear asking questions from
and I'll be the voice **you** hear asking questions from

from the forums Rachel and *Sylvia* are also the people
from the forums Rachel and **Sylv

#### Looking at the top 100 most common pairs of simple single diffs, displayed as:

`('autogen "typo"' <--> 'manual "correction"')`


In [21]:
single_diffs_counter = Counter()
for idx in range(len(transcripts)):
    row = transcripts.iloc[idx]
    new_labels = torch.tensor(row.is_single_diff)
    idxs = torch.where(new_labels == SINGLE_DIFF_LABEL)[0]
    single_diffs_counter.update([
        f"{row.autogen_seq[i]} <--> {row.manual_seq[i]}"
        for i in idxs
    ])                                                

In [22]:
single_diffs_counter.most_common(100)

[('a <--> the', 15),
 ('Jupiter <--> Jupyter', 15),
 ('and <--> in', 11),
 ('main <--> mean', 10),
 ('in <--> and', 10),
 ('the <--> a', 7),
 ('or <--> for', 6),
 ('a <--> an', 6),
 ('is <--> as', 6),
 ("it's <--> is", 5),
 ('and <--> than', 5),
 ('use <--> used', 5),
 ('- <--> minus', 5),
 ("we'll <--> will", 5),
 ('see <--> say', 5),
 ('at <--> it', 5),
 ('Sylvia <--> Sylvain', 4),
 ('first <--> fast', 4),
 ('Pio <--> PIL', 4),
 ('or <--> all', 4),
 ('a <--> of', 3),
 ('of <--> to', 3),
 ('recognize <--> recognise', 3),
 ('ripple <--> REPL', 3),
 ('could <--> can', 3),
 ('recognizing <--> recognising', 3),
 ('then <--> than', 3),
 ('or <--> of', 3),
 ('our <--> R', 3),
 ("you're <--> your", 3),
 ('if <--> of', 3),
 ('and <--> kind', 3),
 ('quite <--> called', 3),
 ('amnesty <--> MNIST', 3),
 ("we're <--> were", 3),
 ('a <--> our', 3),
 ('Apple <--> tuple', 3),
 ('or <--> are', 3),
 ('Kegel <--> Kaggle', 3),
 ('with <--> of', 3),
 ('get <--> git', 3),
 ('faster <--> fastai', 2),
 ('ro